In [1]:
#this file reads the netlist file and try to map the netlist into a graph
import pickle
netlist = 'multiplier_netlist.v'   
library = 'standard_cells_multiplier.v'


#Python Pickle module is used to save objects by serialization. In other words, it can export a dictionary of objects(classes)
#but in the other python file to access it, user needs to load the class definition again. 

In [2]:
#create a class of the standcells
#this class is used to store standardcells information, it has no connection with any verilog netlist file. 
class Std_element:
    def __init__(self,name=None):
        self.input = []
        self.output = []
        self.ports_no = 0  #total number of inputs and outputs
        
    
#make the standard cells into a library and so that the names of standard cell strings can be used as variables
standard_cells = {}

#store the library/(standard cells structure) into the dictionary. The elements in the dictionary are from the class
#Std_element

with open(library) as f:
    for line in f:
        line = line.split()
        if line != []:
            if line[0] == 'module':
                std_name = line[1]
                standard_cells[std_name] = Std_element()   #dictionary里的element都是python的class类型
            elif line[0] == 'input':
                element = line[1].split(',')
                for i in element:
                    standard_cells[std_name].input.append(i)
                    standard_cells[std_name].ports_no += 1
            elif line[0] == 'output':
                element = line[1].split(',')
                for i in element:
                    standard_cells[std_name].output.append(i)
                    standard_cells[std_name].ports_no += 1
                #print element
        

In [3]:
standard_cells

{'AND2X1': <__main__.Std_element instance at 0x049F7300>,
 'AND3X1': <__main__.Std_element instance at 0x049F77D8>,
 'AO22X1': <__main__.Std_element instance at 0x049F71E8>,
 'AOI21X1': <__main__.Std_element instance at 0x049F7260>,
 'AOI22X1': <__main__.Std_element instance at 0x049F7530>,
 'INVX0': <__main__.Std_element instance at 0x049F7698>,
 'NAND2X0': <__main__.Std_element instance at 0x049F7A30>,
 'NAND3X0': <__main__.Std_element instance at 0x049F7850>,
 'NOR2X0': <__main__.Std_element instance at 0x049F76E8>,
 'NOR3X0': <__main__.Std_element instance at 0x049F7580>,
 'OA21X1': <__main__.Std_element instance at 0x049F7760>,
 'OA221X1': <__main__.Std_element instance at 0x049F7418>,
 'OA22X1': <__main__.Std_element instance at 0x049F7620>,
 'OAI21X1': <__main__.Std_element instance at 0x049F73C8>,
 'OAI22X1': <__main__.Std_element instance at 0x049F7990>,
 'OR2X1': <__main__.Std_element instance at 0x049F7490>,
 'OR3X1': <__main__.Std_element instance at 0x049F73F0>,
 'XNOR2X1'

In [4]:
#Node can be considered as wires
class Node:
    def __init__(self,name,weight=None,toggle=None):
        self.name = name
        self.weight = weight     #'weight' is the weight between nodes
        self.toggle = toggle     #'toggle' is a node's toggle count and it is independent to other nodes
        
class adj_list:
    def __init__(self,weight=None,toggle=None):
        self.name = None  #this is the starting point of an adj_list
        self.edge = []    #this dictionary stores the adjacency nodes, inside these nodes are the node name and its value
        self.weight = weight
        self.toggle = toggle
        self.T0 = None
        self.T1 = None
        
    def append(self,name,value=None):
        self.edge.append(name)
  
def info_matching(module, info):
    input_list = []
    output_list = []
    for iter in (standard_cells[gate_type].input):
        vertex = info[info.find('.'+iter)+len(iter)+2:info.find(")",info.find('.'+iter))].replace(" ", "")   #also removes all the white spaces
        #print iter, vertex
        if (vertex.find('\n') != -1):
            vertex = vertex.strip('\n')
        input_list.append(vertex)
    for iter in (standard_cells[gate_type].output):
        vertex = info[info.find('.'+iter)+len(iter)+2:info.find(")",info.find('.'+iter))].replace(" ", "")   #also removes all the white spaces
        if (vertex.find('\n') != -1):
            vertex = vertex.strip('\n')
        output_list.append(vertex)
    return input_list, output_list

In [5]:
#string 'find' here is very import
#want to retrieve string 'egg' between a = "what the {egg} is this"
#use aka = a[a.find('{')+1:a.find('}')]. Then the value of aka will be 'egg'

adjacencylist = {}   #store the Linked_list(for each node) into a dictionary(linkedlist)


#vertex可以理解为wire

incomplete_flag = False  #this indicates if it has already started to scan gate info
collected_info = False
with open(netlist) as f:
    for line in f:
        info = line.split()
        if info != []:
            gate_type = info[0]
            #print line
            if (incomplete_flag):
                #print "ok"
                #pass
                incomplete_flag = False
                updated_line = (partial_line + line.lstrip())
                gate_type = updated_line.split()[0]
                #print updated_line, gate_type
                input_list,output_list = info_matching(gate_type,updated_line)
                collected_info = True
            elif gate_type in standard_cells:    #loop through every gates in the module; it also identifies the line containing gate info
                #print line
                if (info[-1] != ');'):      #means there are other info that is left in the second line
                    partial_line = line
                    incomplete_flag = True
                else:
                    input_list,output_list = info_matching(gate_type,line)
                    #print line
                    #print input_list, output_list
                    collected_info = True
            if (collected_info):
                collected_info = False
                for vertex in (input_list+output_list):
                    if vertex not in adjacencylist:
                        adjacencylist[vertex] = adj_list()
                        adjacencylist[vertex].name = vertex
                for output_vertex in output_list:
                        for input_vertex in input_list:
                            adjacencylist[input_vertex].append(output_vertex)  #这个append是class adj_list里面define的function           
        

In [6]:
adjacencylist['n147'].edge

['n114']

In [7]:
#output the adjacencylist to a pickle file
with open('adjacencylist_multiplier.pickle','wb') as fp:
    pickle.dump(adjacencylist,fp)